In [ ]:
!nvidia-smi -L

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as T
from torchvision.io import read_image
from torchvision.models import ResNet18_Weights

from time import time
import json

In [ ]:
torch.set_float32_matmul_precision("high")
torch.manual_seed(1)

In [ ]:
class Predictor(nn.Module):
    def __init__(self, resnet18):
        super().__init__()
        self.resnet18 = resnet18.eval()
        self.transforms = nn.Sequential(
            T.Resize(
                [
                    256,
                ]
            ),  # We use single int value inside a list due to torchscript type restrictions
            T.CenterCrop(224),
            T.ConvertImageDtype(torch.float),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        with torch.no_grad():
            x = self.transforms(x)
            y_pred = self.resnet18(x)
            return y_pred.argmax(dim=1)

In [ ]:
def pct_diff(t1, t2):
    n = abs(t1 - t2)
    d = abs(t1 + t2) / 2
    return round((n / d) * 100, 3)

In [ ]:
model = torch.hub.load(
    "pytorch/vision:v0.10.0",
    model="resnet18",
    weights=ResNet18_Weights.DEFAULT,
    progress=False,
)
opt_model = torch.compile(model=model, backend="ts_nvfuser")

with open("imagenet_class_index.json", "r") as labels_file:
    labels = json.load(labels_file)

In [ ]:
default_predictor = Predictor(model).to("cuda")
opt_predictor = Predictor(opt_model).to("cuda")

In [ ]:
size = (800, 600)
dog1 = read_image("dog_1.jpg")
dog2 = read_image("dog_2.jpg")
dog1 = T.Resize(size=size)(dog1)
dog2 = T.Resize(size=size)(dog2)
batch = torch.stack([dog1, dog2] * 16).to("cuda")

In [ ]:
default_predictor_result = default_predictor(batch)
for i, pred in enumerate(default_predictor_result):
    print(f"Prediction for Dog {i + 1}: {labels[str(pred.item())]}")

In [ ]:
opt_predictor_result = opt_predictor(batch)
for i, pred in enumerate(opt_predictor_result):
    print(f"Prediction for Dog {i + 1}: {labels[str(pred.item())]}")

In [ ]:
default_start = time()
for i in range(1000):
    default_predictor(batch)
default_stop = time()
default_runtime = default_stop - default_start

In [ ]:
opt_start = time()
for i in range(1000):
    opt_predictor(batch)
opt_stop = time()
opt_runtime = opt_stop - opt_start

In [ ]:
pct_diff(default_runtime, opt_runtime)